## Дизайн AB-теста

Проведение экспериментов — одна из основных и регулярных задач продуктового аналитика. Этот ноутбук рассматривает процесс дизайна и сложности, с которыми можно столкнуться, решая задачу (с фокусом на метрики).

После того, как сформулирована гипотеза, аналитик должен ответить на следующие вопросы:
1. Какая единица рандомизации эксперимента?
2. На какой сегмент будет запущен эксперимент?
3. Какой нужен размер выборки?
4. Как долго будет идти эксперимент?

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import norm

import warnings
warnings.filterwarnings('ignore')

### 1. Единица рандомизации эксперимента

Классические AB-тесты принято запускать с поюзерным разбиением. Однако, такой формат тестирования подходит не для всех гипотез. К другим распространенным подходам относят:
- Свитчбек-тесты — единица рандомизации время или время-пространство;
- Региональные (гео) тесты — единица рандомизации город или регион.

Мы переходим к другим единицам рандомизации, когда проведение поюзерных тестов может исказить результаты из-за сетевого эффекта.

### 2. Сегмент, который попадет в тест

Определяется:
- гипотезой (если она направлена на конкретную группу пользователей);
- местом фичи в продукте (где именно возникает взаимодействие).

Разбиение пользователей можно начинать в момент, когда они сталкиваются с экспериментируемым элементом. Например, если мы тестируем пошаговое оформление покупки вместо единой формы, в эксперимент попадут только пользователи, начавшие процесс оформления.

### 3. Размер выборки

Способ определения размера выборки зависит от критерия, который будем применять к оценке результатов.    
Наиболее распространенный подход — расчёт через MDE. При этом в формуле используются квантильные значения стандартного нормального распределения (z-критерий).

Для определения размера выборки через MDE нужно задать:
- размер эффекта, который мы хотим задетектировать (MDE);
- уровни ошибок I и II рода;
- дисперсию метрики (базовый показатель конверсии для конверсий).

**Для среднего**:

$$
n \;=\; \frac{2\,\sigma^2 \,\big(z_{1-\alpha/2}+z_{1-\beta}\big)^2}{\text{MDE}^2}
$$


**Для доли** (бинарная метрика):

$$
n \;\\=\;\; \frac{2\,p(1-p)\,\big(z_{1-\alpha/2}+z_{1-\beta}\big)^2}{\text{MDE}^2}
$$

- Формулы указаны для групп одинакового размера (50/50) и двустороннего критерия;
- n — размер одной группы.

### 4. Длительность эксперимента

Зависит от размера выборки. Правильный подход — на пред периоде симулировать накопление уникальных пользователей, взаимодействоваших с функционалом, и вернуть число дней, за которое собрался нужный объем. 

Так как мы рассматриваем дизайн с точки зрения используемых метрик, давайте просто возьмем среднее число уникальных пользователей в день по месячным данным. Но важно помнить, что такая оценка очень грубая и может давать заметную погрешность.

### Case Study — дизайн A/B-теста для мобильной игры


**Легенда**   
Представим, что мы аналитики в команде мобильной игры. К нам пришел продакт и просит провести дизайн эксперимента — мы усложняем уровни игры, начиная с 10 уровня, т.к. хотим вырастить средний чек (пользователям будет сложнее пройти дальше, поэтому они будут покупать более дорогие и эффективные атрибуты).

**Гипотеза** — усложнение прохождения игры, начиная с 10 уровня, увеличит средний чек покупки как минимум на 2.5%.

1. Единица рандомизации — поюзерный
2. Сегмент, который попадет в тест — 10+ уровень
3. Размер выборки —

In [2]:
df = pd.read_csv('data/seminar_3_mobile_game.csv')

df.head()

,date,user_id,level,event_id,purchase_amount
0,2025-08-01,1,4,8,NaN
1,2025-08-01,1,4,13,NaN
2,2025-08-01,1,4,12,NaN
3,2025-08-01,1,4,8,NaN
4,2025-08-01,2,1,3,NaN


In [3]:
df.shape

(2168137, 5)

In [4]:
df.user_id.nunique()

20000

In [5]:
df_10_lvl = df[df['level'] >= 10].reset_index(drop=True)

Покупка — event_id = 10

In [6]:
df_10_lvl[df_10_lvl['event_id'] == 10].purchase_amount.mean()

25.872720063200756

In [7]:
def sample_size_mean(metric_series, mde, alpha=0.05, power=0.8, two_sided=True):
    
    var = np.var(metric_series, ddof=1)
    
    if two_sided:
        z_alpha = norm.ppf(1 - alpha/2)
    else:
        z_alpha = norm.ppf(1 - alpha)
    z_beta = norm.ppf(power)
    
    n = (2 * var * (z_alpha + z_beta) ** 2) / (mde ** 2)
    
    return int(np.ceil(n))

In [8]:
sample_size_mean(df_10_lvl[df_10_lvl['event_id'] == 10].purchase_amount, 
                 df_10_lvl[df_10_lvl['event_id'] == 10].purchase_amount.mean() * 0.025, 
                 alpha=0.05, power=0.8, two_sided=True)

8860

Не кажется ли вам, что в дизайне есть что-то подозрительное?

Средний чек — ratio-метрика. Единица расчета (покупка) в ней отличается от единицы рандомизации эксперимента (пользователя). 

А если рассчитать средний чек на пользователя?

In [9]:
df_10_lvl['is_purchase'] = df_10_lvl['event_id'] == 10

In [10]:
df_users_ratio = df_10_lvl.groupby(['user_id'], as_index=False) \
    .agg(sum_amt=('purchase_amount', 'sum'),
         purchases=('is_purchase', 'sum'))

In [11]:
df_users_ratio['avg_check_per_user'] = (df_users_ratio['sum_amt'] / df_users_ratio['purchases']).fillna(0)

In [12]:
df_users_ratio.isna().sum()

user_id               0
sum_amt               0
purchases             0
avg_check_per_user    0
dtype: int64

In [13]:
sample_size_mean(df_users_ratio.avg_check_per_user, 
                 df_users_ratio.avg_check_per_user.mean() * 0.025, 
                 alpha=0.05, power=0.8, two_sided=True)

12349

In [14]:
df_users_ratio.avg_check_per_user.mean()

20.410371413919382

In [15]:
df_10_lvl[df_10_lvl['event_id'] == 10].purchase_amount.mean()

25.872720063200756

Получилась другая метрика :(

**Почему ratio-метрики вообще требуют отдельного внимания?**

Мы применяем критерии, опираясь на их предпосылки. Для t-test и z-распределения — это iid, независимые и одинаково распределённые наблюдения. Когда мы работаем с единицей анализа, отличной от единицы рандомизации, мы нарушаем предпосылку о независимости, ведь несколько покупок одного пользователя будут зависимы. 

Также не стоит забывать, что числитель и знаменатель в Ratio почти всегда зависимы друг от друга (например, мы не получим значение чека в числителе, если покупки в заменателе не случится). Поэтому, для перехода от двух зависимо распределенных величин к третьей величине-отношению, дисперсия новой величины должна учитывать ковариацию между ними.

Что же делать?

### Ratio-метрики

В A/B-тестах без ratio-метрик не обойтись. Благодаря формуле они часто оказываются наиболее удачными:
- устойчивы;
- интерпретируемы.

Поэтому в аналитике разработаны специальные подходы для корректной работы с ratio-метриками.

**1. Переход к поюзерной метрике.**

Перевести ratio на уровень пользователя, чтобы единица анализа совпадала с единицей рандомизации. Например, вместо среднего чека брать ARPU. 

Плюсы: 
- независимость наблюдений, корректный t-test;
- легко интерпретировать дисперсию.
         
Минусы:
- это совсем другая метрика, направление которой может не совпадать с изначальной;
- решение по эксперименту может быть ошибочным (например, если какая-то часть пользователей просто начнет покупать чаще, то мы увидим рост ARPU без роста среднего чека).

**2. Дельта-метод.**

[Оригинальная статья](https://alexdeng.github.io/public/files/kdd2018-dm.pdf) о применении метода в A/B тестировании.    
Подход использует формулу для оценки дисперсиии отношения двух случайных величин:

$$
\mathrm{Var}\!\left(\frac{X}{Y}\right) \;\approx\; 
\frac{1}{\mu_Y^2}\,\mathrm{Var}(X) 
\;+\; \frac{\mu_X^2}{\mu_Y^4}\,\mathrm{Var}(Y) 
\;-\; 2\,\frac{\mu_X}{\mu_Y^3}\,\mathrm{Cov}(X,Y)
$$

тогда

$$
n \;=\; \frac{2 \cdot \mathrm{Var}\!\left(\tfrac{X}{Y}\right)\,\big(z_{1-\alpha/2}+z_{1-\beta}\big)^2}{\text{MDE}^2}
$$

Плюсы: 
- работаем с изначальной метрикой;
- быстрый метод, который хорошо подходит для больших выборок.

Минусы:
- работает на асимптотическом приближении, следовательно может быть не точным при небольших n;
- требует относительно стабильного знаменателя (свитчбек-тестирование может не удовлетворять этому требованию, особенно при семплировании одновременно по маленьким гео и временным слотам).

**3. Линеаризация.**

Переходит от Ratio к новой, линеаризованной метрике (Z) для каждого наблюдения, дисперсию для которой можно рассчитать по формуле для обычного среднего. [Оригинальная статья](https://www.researchgate.net/profile/Roman-Budylin/publication/322969314_Consistent_Transformation_of_Ratio_Metrics_for_Efficient_Online_Controlled_Experiments/links/5b054bbb45851588c6d4a1aa/Consistent-Transformation-of-Ratio-Metrics-for-Efficient-Online-Controlled-Experiments.pdf) от Яндекса.

$$
Z_i \;=\; \frac{\overline X}{\overline Y} \;+\; \frac{1}{\overline Y}\!\left( X_i \;-\; \frac{\overline X}{\overline Y}\, Y_i \right).
$$

тогда

$$
n \;=\; \frac{2 \cdot \mathrm{Var}(Z)\,\big(z_{1-\alpha/2}+z_{1-\beta}\big)^2}{\text{MDE}^2}.
$$

Плюсы: 
- линеаризованная метрика ведет себя как обычная по-юзерная (юзера можно заменить на любой юнит рандомизации), к ней применимы t-test и законы распределения среднего;
- к линеаризованной метрике можно применять любые методы снижения дисперсии (например, CUPED);
- простота реализации.
  
Минусы: 
- работает на асимптотическом приближении, следовательно может быть не точным при небольших n;
- интерпретация может быть непонятна бизнесу;
- подход все еще чувствителен к выбросам в знаменателе (но более устойчив, чем дельта-метод).

Из-за перечисленных преимуществ в России чаще выбирают линеаризацию. В международных материалах чаще встречается дельта-метод, особенно для классических по-юзерных A/B тестов.

In [16]:
def sample_size_ratio_delta(num, den, mde, alpha=0.05, power=0.8, two_sided=True):
    mu_X = num.mean()
    mu_Y = den.mean()
    
    var_X = np.var(num, ddof=1)
    var_Y = np.var(den, ddof=1)
    cov_XY = np.cov(num, den, ddof=1)[0,1]

    var_ratio = (var_X / mu_Y**2
                 - 2 * mu_X * cov_XY / mu_Y**3
                 + (mu_X**2) * var_Y / mu_Y**4)

    if two_sided:
        z_alpha = norm.ppf(1 - alpha/2)
    else:
        z_alpha = norm.ppf(1 - alpha)
    z_beta = norm.ppf(power)

    n = (2 * var_ratio * (z_alpha + z_beta)**2) / (mde**2)
    return int(np.ceil(n))

In [17]:
sample_size_ratio_delta(df_users_ratio.sum_amt, df_users_ratio.purchases, 
                        (df_users_ratio['sum_amt'].sum() / df_users_ratio['purchases'].sum()) * 0.025,  
                        alpha=0.05, power=0.8, two_sided=True)

3699

In [18]:
unique_users_per_day = df.user_id.nunique() / df['date'].nunique()

In [19]:
n = sample_size_ratio_delta(df_users_ratio.sum_amt, df_users_ratio.purchases, 
                        (df_users_ratio['sum_amt'].sum() / df_users_ratio['purchases'].sum()) * 0.025,  
                        alpha=0.05, power=0.8, two_sided=True)

In [20]:
n * 2 /unique_users_per_day

11.4669

### Свитчбек-тесты

При свитчбек-тестировании мы чередуем показ тестового и контрольного варианта, рандомизируя их по времени (слот зависит от тестируемой фичи и специфики продукта, может быть как 5 минут, так и 1 час). Также распространен вариант, в котором переключение происходит не только по времени, но и по пространству (такой подход часто используют в такси, где пространственной единицей становится [гексагон](https://h3geo.org/)).

В контексте курса нам интересны свитчбеки, т.к. почти все метрики для такого дизайна превращаются в Ratio на уровне слота. Даже по-юзерные, ведь теперь в один слот попадет сразу несколько пользователей, а числитель и знаменатель метрики слота приобретут зависимость. Тот же ARPU примет вид:
$$
ARPU_{slot} \;=\; \frac{\text{сумма revenue в слоте}}{\text{число активных пользователей в слоте}}
$$

Поэтому в таких тестах не обойтись без рассмотренных выше подходов.

*Свитчбек-тестирование — отдельная большая тема в рамках онлайн-экспериментов. В работе с ними часто учитывают и равномерность распределения слотов между тестом-контролем в рамках дня недели - часа, и наличие корреляции / сетевого эффекта между слотами (при наличии такой проблемы могут вводить поправку на автокорреляцию / буфферные зоны между переключениями, которые не участвуют в подведении итогов эксперимента). Мы не будем углубляться в специфику, т.к. это курс про метрики, а не про эксперименты. Но имейте в виду, что на рассмотренной реализации, челленджи в свитчбек-тестировании не заканчиваются :)*

В задаче кейса нет предпосылок использовать считчбек. Поэтому давайте возьмем один из датасетов, с котором вы работали дома, с историческими данными сервиса такси, и реализуем дизайн эксперимента для свитчбека 30-минутки и метрики Completion Rate вместе с подходом линеаризации для Ratio.

In [21]:
df_taxi = pd.read_csv('data/hw_2_taxi.csv')

df_taxi.head()

,date,time,order_uid,order_status,passenger_id,vehicle_type,pickup_location,drop_location,ETA,RTA,...,reason_for_cancelling_by_customer,cancelled_rides_by_driver,driver_cancellation_reason,incomplete_rides,incomplete_rides_reason,fare,ride_distance,driver_ratings,customer_rating,payment_method
0,2024-01-01,00:19:34,2a11faf27f77eae8,Completed,CID8362794,Bike,Udyog Vihar,Ambience Mall,9.0,10.8,...,NaN,NaN,NaN,NaN,NaN,99.0,37.98,4.8,4.8,Cash
1,2024-01-01,01:35:18,33ed1f6bad78bdc8,Completed,CID8300238,Go Mini,Basai Dhankot,Madipur,6.0,8.5,...,NaN,NaN,NaN,NaN,NaN,114.0,39.29,4.2,4.1,Uber Wallet
2,2024-01-01,01:37:50,e2fc1fc520e93b85,Cancelled by Driver,CID2030746,Go Sedan,Tughlakabad,Greater Kailash,6.0,7.4,...,NaN,1.0,More than permitted people in there,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-01-01,01:48:03,a130fd507acf7804,Cancelled by Driver,CID3231181,Auto,Palam Vihar,Kherki Daula Toll,5.0,NaN,...,NaN,1.0,Personal & Car related issues,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-01-01,01:49:56,cae2db8689a422fa,Cancelled by Driver,CID3381661,Go Sedan,Narsinghpur,Pulbangash,3.0,6.2,...,NaN,1.0,More than permitted people in there,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df_taxi.order_uid.nunique()

150000

In [23]:
df_taxi.shape

(150000, 22)

In [24]:
df_taxi['date'] = pd.to_datetime(df_taxi['date'])
df_taxi['timestamp'] = pd.to_datetime(df_taxi['date'].dt.strftime('%Y-%m-%d') + ' ' + df_taxi['time'].astype(str))

In [25]:
df_taxi.timestamp.max()

Timestamp('2024-12-30 23:36:11')

Давайте возьмем только декабрьские данные на дизайн

In [26]:
df_taxi_dec = df_taxi[df_taxi['timestamp'] >= pd.Timestamp(2024, 12, 1)].reset_index(drop=True)

In [27]:
df_taxi_dec['is_ride'] = df_taxi_dec['order_status'] == 'Completed'

Получаем датафрейм с агрегированными данными по временным слотам

In [28]:
def get_switchback_units(df, num, den, num_agg, den_agg, time_col, time_slot_min):
    
    df_units = df \
        .set_index(time_col) \
        .groupby(pd.Grouper(freq=f'{time_slot_min}min')) \
        .agg({num:num_agg,
              den:den_agg}) \
        .reset_index() \
        .rename(columns={time_col:'unit_start'})
    
    df_units['unit_end'] = df_units['unit_start'] + pd.Timedelta(minutes=time_slot_min)

    return df_units

In [29]:
df_taxi_dec_time_units = get_switchback_units(df_taxi_dec, 'is_ride', 'order_uid', 'sum', 'nunique', 'timestamp', 30)

df_taxi_dec_time_units.head()

,unit_start,is_ride,order_uid,unit_end
0,2024-12-01 00:00:00,1,2,2024-12-01 00:30:00
1,2024-12-01 00:30:00,0,0,2024-12-01 01:00:00
2,2024-12-01 01:00:00,0,1,2024-12-01 01:30:00
3,2024-12-01 01:30:00,1,1,2024-12-01 02:00:00
4,2024-12-01 02:00:00,0,1,2024-12-01 02:30:00


Линеаризуем метрику

In [32]:
def ratio_linearise(num, den):
    num_mean = np.mean(num)
    den_mean = np.mean(den)
    z_lin = num_mean / den_mean + 1 / den_mean * (num - num_mean / den_mean * den)
    
    return z_lin

In [33]:
z_lin_CR = ratio_linearise(df_taxi_dec_time_units.is_ride, df_taxi_dec_time_units.order_uid)

z_lin_CR[:5]

0    0.593411
1    0.622122
2    0.548991
3    0.666542
4    0.548991
dtype: float64

Используем обычную формулу размера выборки через MDE для среднего, т.к. мы перешли к линеаризованной метрике на юнит. Заложим относительный эффект 4%

In [34]:
sample_size_mean(z_lin_CR, 0.04 * z_lin_CR.mean(), alpha=0.05, power=0.8, two_sided=True)

666

Не забывайте, что размер выборки, который мы получаем в данной формуле, возвращается к нам в единицах рандомизации! В этом примере — в числе 30-минуток. Поэтому, для расчета длительности теста в днях, будем делить полученное число на количество 30-минуток за день

In [35]:
n_group = sample_size_mean(z_lin_CR, 0.04 * z_lin_CR.mean(), alpha=0.05, power=0.8, two_sided=True)

n_group * 2 / (24 * 2)

27.75

Для детектирования относительного эффекта в 4% для метрики Completion Rate для свитчбека 30-минутки нам потребуется 28 дней